In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

!pip install uv




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 55.8 MB/s eta 0:00:00


In [4]:




!uv pip install langchain langchain-community langchain-huggingface sentence-transformers faiss-cpu

Using Python 3.11.13 environment at: /usr
Resolved 83 packages in 644ms
⠙ Preparing packages... (0/20)
⠙ Preparing packages... (0/20)
⠙ Preparing packages... (0/20)
marshmallow              ------------------------------     0 B/49.69 KiB
⠙ Preparing packages... (0/20)
marshmallow              ------------------------------     0 B/49.69 KiB
⠙ Preparing packages... (0/20)
marshmallow              ------------------------------     0 B/49.69 KiB
⠙ Preparing packages... (0/20)
pydantic-settings        ------------------------------     0 B/44.17 KiB
marshmallow              ------------------------------     0 B/49.69 KiB
⠙ Preparing packages... (0/20)
pydantic-settings        ------------------------------     0 B/44.17 KiB
marshmallow              ------------------------------     0 B/49.69 KiB
⠙ Preparing packages... (0/20)
pydantic-settings        ------------------------------     0 B/44.17 KiB
marshmallow              ------------------------------     0 B/49.69 KiB
⠙ Preparing pa

In [5]:
!uv pip install unstructured

Using Python 3.11.13 environment at: /usr
Resolved 51 packages in 330ms
⠙ Preparing packages... (0/12)
⠙ Preparing packages... (0/12)
⠙ Preparing packages... (0/12)
filetype             ------------------------------     0 B/19.50 KiB
⠙ Preparing packages... (0/12)
filetype             ------------------------------ 14.84 KiB/19.50 KiB
⠙ Preparing packages... (0/12)
filetype             ------------------------------ 14.84 KiB/19.50 KiB
⠙ Preparing packages... (0/12)
filetype             ------------------------------ 14.84 KiB/19.50 KiB
⠙ Preparing packages... (0/12)
filetype             ------------------------------ 14.84 KiB/19.50 KiB
olefile              ------------------------------     0 B/111.88 KiB
⠙ Preparing packages... (0/12)
filetype             ------------------------------ 14.84 KiB/19.50 KiB
olefile              ------------------------------ 14.93 KiB/111.88 KiB
⠙ Preparing packages... (0/12)
filetype             ------------------------------ 14.84 KiB/19.50 KiB
ole

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [7]:
RAW_DATA_PATH="/content/drive/MyDrive/BARKAI/raw_data"
loader = DirectoryLoader(RAW_DATA_PATH, glob="**/*.txt", show_progress=True)
documents = loader.load()

100%|██████████| 17/17 [01:00<00:00,  3.54s/it]


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=150)
chunks = text_splitter.split_documents(documents)


In [10]:
import torch
embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [11]:
db = FAISS.from_documents(chunks, embedding_model)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [13]:
import os
VECTOR_STORE_PATH = "/content/drive/My Drive/BARKAI/vector_store"
os.makedirs(VECTOR_STORE_PATH, exist_ok=True)
db.save_local(VECTOR_STORE_PATH)